# 라이브러리

In [1]:
import os

import pandas as pd

# 파일 불러오기

In [2]:
DATA_PATH = "./data/register_car/"
EXCEL_LIST = os.listdir(DATA_PATH)

EXCEL_LIST

['자동차등록현황보고_자동차등록대수현황 시도별 202505.xlsx', '자동차등록현황보고_자동차등록대수현황 시도별 202506.xlsx']

# 데이터 프레임으로 확인

In [3]:
df = pd.read_excel(DATA_PATH + EXCEL_LIST[0])

df

c:\khs\github\SKNW10-1st_project\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,자동차등록현황보고(Total Registered Motor Vehicles) / 자동차등록대수현황 시도별 (201101 ~ 202506),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,다운로드 시간 : 2025-08-01 09:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,검색기간 : 202505 ~ 202505,검색분야 : 종합일반,작성주기 : 매월,단위 : 대,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,월(Monthly),시도명,시군구,승용,승용,승용,승용,승합,승합,승합,...,화물,화물,특수,특수,특수,특수,총계,총계,총계,총계
4,월(Monthly),시도명,시군구,관용,자가용,영업용,계,관용,자가용,영업용,...,영업용,계,관용,자가용,영업용,계,관용,자가용,영업용,계
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2025-05,전북,전주시 덕진구,158,"151,019","3,074","154,251",103,"3,260",933,...,"2,614","21,650",14,571,403,988,427,"173,734","7,024","181,185"
273,2025-05,전북,전주시 완산구,342,"136,966","2,005","139,313",227,"2,928",539,...,"1,532","17,502",23,359,252,634,803,"156,012","4,328","161,143"
274,2025-05,전북,정읍시,107,"41,491","1,510","43,108",111,"1,490",198,...,895,"15,947",8,254,122,384,388,"58,125","2,725","61,238"
275,2025-05,전북,진안군,110,"9,817",91,"10,018",53,399,95,...,107,"5,272",15,58,16,89,289,"15,328",309,"15,926"


In [36]:
test_index = df[(df.iloc[:, 1] == "전북") & (df.iloc[:, 2] == "전주시 덕진구")].index

In [40]:
df.loc[test_index, (df.iloc[3, :] == "승용") & (df.iloc[4, :] == "자가용")].values

array([['151,019']], dtype=object)

In [8]:
df.iloc[:, 1]

0                          NaN
1      검색기간  : 202505 ~ 202505
2                          NaN
3                          시도명
4                          시도명
                ...           
272                         전북
273                         전북
274                         전북
275                         전북
276                         전북
Name: Unnamed: 1, Length: 277, dtype: object

In [5]:
import pandas as pd


# 3, 4번째 줄 (index=3, 4)에서 열 이름 추출
multi_header = df.iloc[3:5]
column_tuples = list(zip(*multi_header.values))  # 전치(transpose) 후 튜플로 만듦

# 새 헤더 지정
new_columns = []
for level1, level2 in column_tuples:
    if pd.isna(level1):
        new_columns.append(level2)
    elif pd.isna(level2):
        new_columns.append(level1)
    else:
        new_columns.append(f"{level1}_{level2}")

# 5번째 줄부터 본 데이터 시작
df_data = df.iloc[5:].copy()
df_data.columns = new_columns

# 앞부분 월, 시도명, 시군구 추출
id_vars = ['2025-05', '시도명', '시군구']
value_vars = [col for col in df_data.columns if col not in id_vars]

# 데이터를 long format으로 melt
df_melted = df_data.melt(id_vars=id_vars, value_vars=value_vars,
                         var_name="차종_용도", value_name="등록수")

# 차종과 용도를 나눔
df_melted[['차종', '용도']] = df_melted['차종_용도'].str.split('_', expand=True)
df_melted.drop(columns=['차종_용도'], inplace=True)

# 결과 정렬 및 필요시 숫자형으로 변환
df_melted['등록수'] = pd.to_numeric(df_melted['등록수'], errors='coerce')
df_melted = df_melted.dropna(subset=['등록수'])

# 최종 결과 확인
print(df_melted.head())

KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['2025-05', '시도명', '시군구']"

In [6]:
column_tuples

[('월(Monthly)', '월(Monthly)'),
 ('시도명', '시도명'),
 ('시군구', '시군구'),
 ('승용', '관용'),
 ('승용', '자가용'),
 ('승용', '영업용'),
 ('승용', '계'),
 ('승합', '관용'),
 ('승합', '자가용'),
 ('승합', '영업용'),
 ('승합', '계'),
 ('화물', '관용'),
 ('화물', '자가용'),
 ('화물', '영업용'),
 ('화물', '계'),
 ('특수', '관용'),
 ('특수', '자가용'),
 ('특수', '영업용'),
 ('특수', '계'),
 ('총계', '관용'),
 ('총계', '자가용'),
 ('총계', '영업용'),
 ('총계', '계')]

In [4]:
# 시도_1
list(df.iloc[:, 1].unique())[3:]

['서울',
 '부산',
 '대구',
 '인천',
 '광주',
 '대전',
 '울산',
 '세종',
 '경기',
 '충북',
 '충남',
 '전남',
 '경북',
 '경남',
 '제주',
 '강원',
 '전북']

In [5]:
# 시도_2
list(df.iloc[:, 1])[5:]

['서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '서울',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '부산',
 '대구',
 '대구',
 '대구',
 '대구',
 '대구',
 '대구',
 '대구',
 '대구',
 '대구',
 '대구',
 '인천',
 '인천',
 '인천',
 '인천',
 '인천',
 '인천',
 '인천',
 '인천',
 '인천',
 '인천',
 '인천',
 '광주',
 '광주',
 '광주',
 '광주',
 '광주',
 '광주',
 '대전',
 '대전',
 '대전',
 '대전',
 '대전',
 '대전',
 '울산',
 '울산',
 '울산',
 '울산',
 '울산',
 '울산',
 '세종',
 '세종',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '경기',
 '충북',
 '충북',
 '충북',
 '충북',
 '충북',
 '충북',
 '충북',
 '충북',
 '충북',
 '충북',
 '충북',
 '충북',
 '충북',

In [6]:
# 시군구
list(df.iloc[:, 2].unique())[3:]

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구',
 '계',
 '금정구',
 '기장군',
 '남구',
 '동구',
 '동래구',
 '부산진구',
 '북구',
 '사상구',
 '사하구',
 '서구',
 '수영구',
 '연제구',
 '영도구',
 '해운대구',
 '군위군',
 '달서구',
 '달성군',
 '수성구',
 '강화군',
 '계양구',
 '남동구',
 '미추홀구',
 '부평구',
 '연수구',
 '옹진군',
 '광산구',
 '대덕구',
 '유성구',
 '울주군',
 '세종특별자치시',
 '가평군',
 '고양시 덕양구',
 '고양시 일산동구',
 '고양시 일산서구',
 '과천시',
 '광명시',
 '광주시',
 '구리시',
 '군포시',
 '김포시',
 '남양주시',
 '동두천시',
 '부천시',
 '부천시 소사구',
 '부천시 오정구',
 '부천시 원미구',
 '성남시 분당구',
 '성남시 수정구',
 '성남시 중원구',
 '수원시 권선구',
 '수원시 영통구',
 '수원시 장안구',
 '수원시 팔달구',
 '시흥시',
 '안산시 단원구',
 '안산시 상록구',
 '안성시',
 '안양시 동안구',
 '안양시 만안구',
 '양주시',
 '양평군',
 '여주시',
 '연천군',
 '오산시',
 '용인시 기흥구',
 '용인시 수지구',
 '용인시 처인구',
 '의왕시',
 '의정부시',
 '이천시',
 '파주시',
 '평택시',
 '포천시',
 '하남시',
 '화성시',
 '괴산군',
 '단양군',
 '보은군',
 '영동군',
 '옥천군',
 '음성군',
 '제천시',
 '증평군',
 '진천군',
 '청원군',
 '청주시 상당구',
 '청주시 서원구',


In [7]:
list(df.iloc[:, 2])[5:]

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구',
 '계',
 '강서구',
 '금정구',
 '기장군',
 '남구',
 '동구',
 '동래구',
 '부산진구',
 '북구',
 '사상구',
 '사하구',
 '서구',
 '수영구',
 '연제구',
 '영도구',
 '중구',
 '해운대구',
 '계',
 '군위군',
 '남구',
 '달서구',
 '달성군',
 '동구',
 '북구',
 '서구',
 '수성구',
 '중구',
 '계',
 '강화군',
 '계양구',
 '남동구',
 '동구',
 '미추홀구',
 '부평구',
 '서구',
 '연수구',
 '옹진군',
 '중구',
 '계',
 '광산구',
 '남구',
 '동구',
 '북구',
 '서구',
 '계',
 '대덕구',
 '동구',
 '서구',
 '유성구',
 '중구',
 '계',
 '남구',
 '동구',
 '북구',
 '울주군',
 '중구',
 '계',
 '세종특별자치시',
 '계',
 '가평군',
 '고양시 덕양구',
 '고양시 일산동구',
 '고양시 일산서구',
 '과천시',
 '광명시',
 '광주시',
 '구리시',
 '군포시',
 '김포시',
 '남양주시',
 '동두천시',
 '부천시',
 '부천시 소사구',
 '부천시 오정구',
 '부천시 원미구',
 '성남시 분당구',
 '성남시 수정구',
 '성남시 중원구',
 '수원시 권선구',
 '수원시 영통구',
 '수원시 장안구',
 '수원시 팔달구',
 '시흥시',
 '안산시 단원구',
 '안산시 상록구',
 '안성시',
 '안양시 동안구',
 '안양시 만안구',
 '양주시',
 '양평군',
 '여주시',
 '연천군',
 '오산시',
 '용인시 기흥구',
 '용

In [8]:
# 시도 시군구
list(df.iloc[:, 1] + " " + df.iloc[:, 2])[5:]

['서울 강남구',
 '서울 강동구',
 '서울 강북구',
 '서울 강서구',
 '서울 관악구',
 '서울 광진구',
 '서울 구로구',
 '서울 금천구',
 '서울 노원구',
 '서울 도봉구',
 '서울 동대문구',
 '서울 동작구',
 '서울 마포구',
 '서울 서대문구',
 '서울 서초구',
 '서울 성동구',
 '서울 성북구',
 '서울 송파구',
 '서울 양천구',
 '서울 영등포구',
 '서울 용산구',
 '서울 은평구',
 '서울 종로구',
 '서울 중구',
 '서울 중랑구',
 '서울 계',
 '부산 강서구',
 '부산 금정구',
 '부산 기장군',
 '부산 남구',
 '부산 동구',
 '부산 동래구',
 '부산 부산진구',
 '부산 북구',
 '부산 사상구',
 '부산 사하구',
 '부산 서구',
 '부산 수영구',
 '부산 연제구',
 '부산 영도구',
 '부산 중구',
 '부산 해운대구',
 '부산 계',
 '대구 군위군',
 '대구 남구',
 '대구 달서구',
 '대구 달성군',
 '대구 동구',
 '대구 북구',
 '대구 서구',
 '대구 수성구',
 '대구 중구',
 '대구 계',
 '인천 강화군',
 '인천 계양구',
 '인천 남동구',
 '인천 동구',
 '인천 미추홀구',
 '인천 부평구',
 '인천 서구',
 '인천 연수구',
 '인천 옹진군',
 '인천 중구',
 '인천 계',
 '광주 광산구',
 '광주 남구',
 '광주 동구',
 '광주 북구',
 '광주 서구',
 '광주 계',
 '대전 대덕구',
 '대전 동구',
 '대전 서구',
 '대전 유성구',
 '대전 중구',
 '대전 계',
 '울산 남구',
 '울산 동구',
 '울산 북구',
 '울산 울주군',
 '울산 중구',
 '울산 계',
 '세종 세종특별자치시',
 '세종 계',
 '경기 가평군',
 '경기 고양시 덕양구',
 '경기 고양시 일산동구',
 '경기 고양시 일산서구',
 '경기 과천시',
 '경기 광명시',
 '경기 광주시',
 '경기 구리시',
 '경기 

In [9]:
# 차종, 용도
## 차종
print(list(df.iloc[3, :].unique())[3:])

## 용도
print(list(df.iloc[4, :].unique())[3:])

['승용', '승합', '화물', '특수', '총계']
['관용', '자가용', '영업용', '계']


In [10]:
columns = ["시도", "시군구", "차종", "용도"]

sido = list(df.iloc[:, 1])[5:]
sigungu = list(df.iloc[:, 2])[5:]
car_type = list(df.iloc[3, :].unique())[3:]
car_purpose = list(df.iloc[4, :].unique())[3:]

In [11]:
df = pd.DataFrame(
    {
        columns[0]: sido,
        columns[1]: sigungu,
    }
)

df.head()

,시도,시군구
0,서울,강남구
1,서울,강동구
2,서울,강북구
3,서울,강서구
4,서울,관악구


In [ ]:
# car_register_meta table
for _, v in df.iterrows():
    for type in car_type:
        for purpose in car_purpose:
            text = list(v.values)
            text.append(type)
            text.append(purpose)
            print(text)
            break
        break
    break

['서울', '강남구', '승용', '관용']


In [13]:
pd.DataFrame([['a','b','c'], ['x','y','z']])

,0,1,2
0,a,b,c
1,x,y,z


In [ ]:
test_list = []
test_list.append(['a','b','c'])
test_list.append(['d','e','f'])
print(test_list)

[['a', 'b', 'c'], ['d', 'e', 'f']]


In [16]:
pd.DataFrame(test_list)

,0,1,2
0,a,b,c
1,d,e,f


In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "year": [2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024],
        "month": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        "value": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200]
    }
)

0     2024
1     2024
2     2024
3     2024
4     2024
5     2024
6     2024
7     2024
8     2024
9     2024
10    2024
11    2024
Name: year, dtype: int64

In [5]:
list(df["year"]) + list(df["month"])

[2024,
 2024,
 2024,
 2024,
 2024,
 2024,
 2024,
 2024,
 2024,
 2024,
 2024,
 2024,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12]